<a href="https://colab.research.google.com/github/ShreyMhatre/nanoGPT/blob/main/nanoGPT_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nanoGPT from scratch

Only *validation.csv* is being used to reduce training time.

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "validation.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "thedevastator/tinystories-narrative-classification",
  file_path,
)

print("First 5 records:", df.head())

/tmp/ipython-input-1-625683540.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 5.74M/5.74M [00:00<00:00, 101MB/s]

Extracting zip of validation.csv...


First 5 records:                                                 text
0  Spot. Spot saw the shiny car and said, "Wow, K...
1  Once upon a time, in a big forest, there lived...
2  Once upon a time, in a small yard, there was a...
3  Once upon a time, there was a thoughtful girl ...
4  Once upon a time, there was a kind farmer. He ...


In [ ]:
df.to_csv('input.txt', header=False, index=False)

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of dataset in characters", len(text))

length of dataset in characters 19349772


In [ ]:
print(text[:1000])

"Spot. Spot saw the shiny car and said, ""Wow, Kitty, your car is so bright and clean!"" Kitty smiled and replied, ""Thank you, Spot. I polish it every day.""

After playing with the car, Kitty and Spot felt thirsty. They found a small pond with clear water. They drank the water and felt very happy. They played together all day and became best friends."
"Once upon a time, in a big forest, there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, rocks, and hills. One day, Roxy found an icy hill. She had never seen anything like it before. It was shiny and cold, and she wanted to climb it.

Roxy tried to climb the icy hill, but it was very slippery. She tried again and again, but she kept falling down. Roxy was sad. She wanted to climb the icy hill so much. Then, she saw a little bird named Billy. Billy saw that Roxy was sad and asked, ""Why are you sad, Roxy?""

Roxy told Billy about the icy hill and how she couldn't climb it. Billy said, ""I have an idea! Let's find

In [ ]:
# All the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$&'()*+,-./0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz¦©­±´ÂÃâðœŠŸŽ˜“”‹€™
101


In [ ]:
# create mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[63, 64, 64, 1, 75, 63, 60, 73, 60]
hii there


### Encoding text dataset into a torch.Tensor

In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([19349772]) torch.int64
tensor([ 3, 48, 71, 70, 75, 14,  1, 48, 71, 70, 75,  1, 74, 56, 78,  1, 75, 63,
        60,  1, 74, 63, 64, 69, 80,  1, 58, 56, 73,  1, 56, 69, 59,  1, 74, 56,
        64, 59, 12,  1,  3,  3, 52, 70, 78, 12,  1, 40, 64, 75, 75, 80, 12,  1,
        80, 70, 76, 73,  1, 58, 56, 73,  1, 64, 74,  1, 74, 70,  1, 57, 73, 64,
        62, 63, 75,  1, 56, 69, 59,  1, 58, 67, 60, 56, 69,  2,  3,  3,  1, 40,
        64, 75, 75, 80,  1, 74, 68, 64, 67, 60, 59,  1, 56, 69, 59,  1, 73, 60,
        71, 67, 64, 60, 59, 12,  1,  3,  3, 49, 63, 56, 69, 66,  1, 80, 70, 76,
        12,  1, 48, 71, 70, 75, 14,  1, 38,  1, 71, 70, 67, 64, 74, 63,  1, 64,
        75,  1, 60, 77, 60, 73, 80,  1, 59, 56, 80, 14,  3,  3,  0,  0, 30, 61,
        75, 60, 73,  1, 71, 67, 56, 80, 64, 69, 62,  1, 78, 64, 75, 63,  1, 75,
        63, 60,  1, 58, 56, 73, 12,  1, 40, 64, 75, 75, 80,  1, 56, 69, 59,  1,
        48, 71, 70, 75,  1, 61, 60, 67, 75,  1, 75, 63, 64, 73, 74, 75, 80, 14,
     

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([ 3, 48, 71, 70, 75, 14,  1, 48, 71])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([3]) the target: 48
when input is tensor([ 3, 48]) the target: 71
when input is tensor([ 3, 48, 71]) the target: 70
when input is tensor([ 3, 48, 71, 70]) the target: 75
when input is tensor([ 3, 48, 71, 70, 75]) the target: 14
when input is tensor([ 3, 48, 71, 70, 75, 14]) the target: 1
when input is tensor([ 3, 48, 71, 70, 75, 14,  1]) the target: 48
when input is tensor([ 3, 48, 71, 70, 75, 14,  1, 48]) the target: 71


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independant sequences will we process in parallel?
block_size = 8 # what is the maximum context length for prediction?

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    traget = yb[b,t]
    print(f"when input is {context.tolist()} the target: {traget}")

inputs:
torch.Size([4, 8])
tensor([[73, 70, 68,  1, 57, 60, 63, 64],
        [70, 76, 75, 14,  0,  0, 49, 63],
        [44, 69, 58, 60,  1, 76, 71, 70],
        [56, 69, 75, 64, 58, 64, 71, 56]])
targets:
torch.Size([4, 8])
tensor([[70, 68,  1, 57, 60, 63, 64, 69],
        [76, 75, 14,  0,  0, 49, 63, 60],
        [69, 58, 60,  1, 76, 71, 70, 69],
        [69, 75, 64, 58, 64, 71, 56, 75]])
----
when input is [73] the target: 70
when input is [73, 70] the target: 68
when input is [73, 70, 68] the target: 1
when input is [73, 70, 68, 1] the target: 57
when input is [73, 70, 68, 1, 57] the target: 60
when input is [73, 70, 68, 1, 57, 60] the target: 63
when input is [73, 70, 68, 1, 57, 60, 63] the target: 64
when input is [73, 70, 68, 1, 57, 60, 63, 64] the target: 69
when input is [70] the target: 76
when input is [70, 76] the target: 75
when input is [70, 76, 75] the target: 14
when input is [70, 76, 75, 14] the target: 0
when input is [70, 76, 75, 14, 0] the target: 0
when input is [70

In [ ]:
print(xb)

tensor([[73, 70, 68,  1, 57, 60, 63, 64],
        [70, 76, 75, 14,  0,  0, 49, 63],
        [44, 69, 58, 60,  1, 76, 71, 70],
        [56, 69, 75, 64, 58, 64, 71, 56]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # since cross_entropy requires C in 2nd position
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 101])
tensor(5.3060, grad_fn=<NllLossBackward0>)

UX“'h.h5L:hTGR'd$C04Ž™vv.s”V,r˜SlJ<p™fmârWph:‹XG'n"g<;Ž#D”f¦83WNvAD6f©a?ff­rE7bQnPjA'ðD4#uÂ
ql&!Â:lg


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.235708475112915


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))




""
We kit kit chad cheapule t thululy bire foushe owand.

Timar hete Lin a 
One.
Joul t!"It Thidou


## Self-attention

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
# we want x[b,t] = mean_{i<=t} x[b,i]
# bow = bag of words
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [ ]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1, device=None, dtype=None):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
      xmean = x.mean(1, keepdim=True) # batch mean
      xvar = x.var(1, keepdim=True) # batch variance
      xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
      self.out = self.gamma * xhat + self.beta
      return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch of 32 samples of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))